In [1]:
# Cell 1 - Setup
import os, json, math, random, textwrap
from collections import deque
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
sns.set(style="whitegrid")

# Create folders
os.makedirs('/content/pipeline', exist_ok=True)
os.makedirs('/content/pipeline/plots', exist_ok=True)
os.makedirs('/content/pipeline/out', exist_ok=True)

print("Folders created: /content/pipeline, /content/pipeline/plots, /content/pipeline/out")


Folders created: /content/pipeline, /content/pipeline/plots, /content/pipeline/out


In [2]:
# Cell 2 - Pipeline simulator (5-stage) with forwarding, stalling, branch flush

# Instruction representation
# Simple ISA: op, rd, rs1, rs2, imm, is_branch, writes_reg
from dataclasses import dataclass

@dataclass
class Instr:
    pc: int
    op: str            # 'ADD','SUB','LW','SW','BEQ','NOP'
    rd: str = None
    rs1: str = None
    rs2: str = None
    imm: int = 0
    writes: bool = False
    is_branch: bool = False

# Example register names: 'x1'..'x31'
REGS = [f'x{i}' for i in range(32)]

# Pipeline stages: IF, ID, EX, MEM, WB
STAGE_NAMES = ['IF','ID','EX','MEM','WB']

class PipelineSim:
    def __init__(self, instr_list, use_forwarding=True, use_stall=True, branch_predict_taken=False):
        self.instrs = {ins.pc: ins for ins in instr_list}
        self.pc = min(self.instrs.keys())
        self.cycle = 0
        self.registers = {r:0 for r in REGS}
        self.mem = {}   # memory map
        self.pipeline = {s: None for s in STAGE_NAMES}
        self.finished = []
        self.trace = []  # per cycle snapshot of pipeline
        self.use_forwarding = use_forwarding
        self.use_stall = use_stall
        self.branch_predict_taken = branch_predict_taken
        self.stall_next = False
        self.flush_next_pc = None
        self.max_cycles = 1000

    def fetch(self):
        # IF stage: fetch instruction at pc (or NOP if none)
        ins = self.instrs.get(self.pc, None)
        if ins is None:
            return Instr(self.pc, 'NOP', writes=False)
        return ins

    def reg_written_by(self, instr):
        return instr and instr.writes and instr.rd

    def reg_read_by(self, instr):
        reads = set()
        if instr is None:
            return reads
        if instr.rs1: reads.add(instr.rs1)
        if instr.rs2: reads.add(instr.rs2)
        return reads

    def hazard_detect_and_stall(self):
        # RAW hazard detection: if ID stage reads a reg that is written by EX or MEM and forwarding not available, stall.
        id_ins = self.pipeline['ID']
        if id_ins is None or id_ins.op == 'NOP':
            return False
        # register reads in ID stage
        reads = self.reg_read_by(id_ins)
        # producers
        ex_ins = self.pipeline['EX']
        mem_ins = self.pipeline['MEM']
        wb_ins = self.pipeline['WB']
        # if forwarding available, we allow certain hazards
        for prod in [ex_ins, mem_ins]:
            if prod and prod.writes and prod.rd:
                if prod.rd in reads:
                    # If forwarding enabled and prod in EX stage - forwarding from EX to ID->EX allowed?
                    if self.use_forwarding:
                        # For simplicity we assume forwarding from EX and MEM sufficient for ALU ops; loads in EX can't forward until MEM.
                        if prod.op == 'LW' and prod is ex_ins:
                            # load-use hazard: cannot forward this cycle -> must stall
                            if self.use_stall:
                                return True
                        else:
                            # forwarding would resolve
                            continue
                    else:
                        # no forwarding -> stall
                        if self.use_stall:
                            return True
        return False

    def step(self):
        self.cycle += 1
        # writeback stage completes
        # Move pipeline backward (WB done)
        finished_instr = self.pipeline['WB']
        if finished_instr and finished_instr.op != 'NOP':
            self.finished.append(finished_instr)
        # Advance stages: WB <= MEM, MEM <= EX, EX <= ID, ID <= IF, IF <= new fetch (subject to stalls)
        # But must implement stall logic
        # 1. detect hazard: if stall needed, insert NOP into EX (bubble) and keep ID same, do not advance IF
        need_stall = self.hazard_detect_and_stall()
        # 2. handle branch in EX: if branch resolved and mispredicted -> flush IF and ID (insert NOPs) and set PC
        # We'll assume branch is resolved in EX and uses branch_predict_taken flag to predict
        # Propagate from MEM->WB
        self.pipeline['WB'] = self.pipeline['MEM']
        self.pipeline['MEM'] = self.pipeline['EX']
        # EX stage: if a stall, insert NOP
        if need_stall:
            # Insert bubble in EX, keep ID and freeze IF (no new fetch)
            self.pipeline['EX'] = Instr(-1, 'NOP', writes=False)
        else:
            self.pipeline['EX'] = self.pipeline['ID']
            # ID will be overwritten by IF below
        # ID <= IF (unless branch flush or stall)
        if not need_stall:
            self.pipeline['ID'] = self.pipeline['IF']
        # IF stage: fetch next instruction based on PC unless branch taken/flush
        # If previous cycle had a flush request (due to mispredict), set PC accordingly
        if self.flush_next_pc is not None:
            self.pc = self.flush_next_pc
            self.flush_next_pc = None
            self.pipeline['IF'] = self.fetch()
            # increment pc for next fetch
            self.pc += 4
        else:
            if need_stall:
                # freeze IF (do not advance PC), but pipeline['IF'] remains same (simulate stalled fetch)
                # For simplicity keep IF as previous fetch (or fill with NOP if None)
                # If IF was None, put NOP
                if self.pipeline['IF'] is None:
                    self.pipeline['IF'] = Instr(self.pc, 'NOP', writes=False)
            else:
                self.pipeline['IF'] = self.fetch()
                self.pc += 4

        # Branch resolution: when an instr in EX is a branch, resolve here
        ex_ins = self.pipeline['EX']
        if ex_ins and ex_ins.op == 'BEQ':
            # simple branch: compare registers (we use register file values)
            rs1 = ex_ins.rs1; rs2 = ex_ins.rs2
            take = (self.registers.get(rs1, 0) == self.registers.get(rs2, 0))
            predicted = self.branch_predict_taken
            if predicted != take:
                # mispredict: flush IF and ID and set PC to correct target
                # target = PC + imm (imm is multiples of 4)
                target_pc = ex_ins.pc + ex_ins.imm
                self.flush_next_pc = target_pc
                # place NOPs into IF and ID next cycle handled via flush_next_pc logic
        # Record snapshot
        snap = {s: (self.pipeline[s].op if self.pipeline[s] else None,
                    (self.pipeline[s].rd if self.pipeline[s] else None)) for s in STAGE_NAMES}
        self.trace.append((self.cycle, snap))
        # stop condition: if all pipeline slots are NOP or None and pc beyond last instr
        active = any((self.pipeline[s] and self.pipeline[s].op != 'NOP') for s in STAGE_NAMES)
        if not active and self.pc > max(self.instrs.keys()):
            return False  # finished
        if self.cycle > self.max_cycles:
            return False
        return True

    def run(self, max_cycles=1000):
        self.max_cycles = max_cycles
        # initialize pipeline with NOPs
        for s in STAGE_NAMES:
            self.pipeline[s] = Instr(-1,'NOP',writes=False)
        keep_running = True
        while keep_running:
            keep_running = self.step()
        return self.trace

    def compute_cpi(self):
        total_instructions = len([i for i in self.finished if i.op != 'NOP'])
        total_cycles = self.cycle
        cpi = total_cycles / total_instructions if total_instructions>0 else float('inf')
        return {'total_instructions': total_instructions, 'total_cycles': total_cycles, 'CPI': cpi}


In [3]:
# Cell 3 - Example instruction sequences for hazard simulation
# We'll use simple example ISA semantics (not executing values except for branch compare)
def make_instr(pc, op, rd=None, rs1=None, rs2=None, imm=0):
    writes = op in ('ADD','SUB','LW')
    is_branch = (op == 'BEQ')
    return Instr(pc=pc, op=op, rd=rd, rs1=rs1, rs2=rs2, imm=imm, writes=writes, is_branch=is_branch)

# Example 1: RAW hazard (load-use)
# LW x1, 0(x0)
# ADD x2, x1, x3  <-- dependent on x1 (load-use) -> needs stall if no forwarding from MEM
# ADD x4, x2, x5
seq1 = [
    make_instr(0, 'LW', rd='x1', rs1='x0'),
    make_instr(4, 'ADD', rd='x2', rs1='x1', rs2='x3'),
    make_instr(8, 'ADD', rd='x4', rs1='x2', rs2='x5'),
]

# Example 2: ALU forwarding case
seq2 = [
    make_instr(0, 'ADD', rd='x1', rs1='x2', rs2='x3'),
    make_instr(4, 'SUB', rd='x4', rs1='x1', rs2='x5'), # can forward from EX/MEM
    make_instr(8, 'ADD', rd='x6', rs1='x4', rs2='x7'),
]

# Example 3: Control hazard (branch)
seq3 = [
    make_instr(0, 'ADD', rd='x1', rs1='x0', rs2='x0'),
    make_instr(4, 'BEQ', rs1='x1', rs2='x0', imm=12),  # branch to pc 16 if equal
    make_instr(8, 'ADD', rd='x2', rs1='x3', rs2='x4'),  # in delay-of-branch path
    make_instr(12,'SUB', rd='x5', rs1='x6', rs2='x7'),
    make_instr(16,'ADD', rd='x8', rs1='x9', rs2='x10'),
]

# Example 4: Multiple read-after-write chain
seq4 = [
    make_instr(0, 'ADD', rd='x1', rs1='x2', rs2='x3'),
    make_instr(4, 'ADD', rd='x2', rs1='x1', rs2='x4'),
    make_instr(8, 'ADD', rd='x3', rs1='x2', rs2='x5'),
    make_instr(12,'ADD', rd='x4', rs1='x3', rs2='x6'),
]

examples = {'load_use': seq1, 'alu_forward': seq2, 'branch_case': seq3, 'chain': seq4}
print("Prepared example sequences:", list(examples.keys()))


Prepared example sequences: ['load_use', 'alu_forward', 'branch_case', 'chain']


In [4]:
# Cell 4 - Run examples and save timeline tables
def run_and_save(seq, name, forwarding=True, stalling=True, predict_taken=False):
    sim = PipelineSim(seq, use_forwarding=forwarding, use_stall=stalling, branch_predict_taken=predict_taken)
    trace = sim.run(max_cycles=200)
    # Build DataFrame: rows=cycle, cols=stages
    rows = []
    for cycle, snap in trace:
        row = {'cycle': cycle}
        for s in STAGE_NAMES:
            op, rd = snap.get(s, (None,None))
            row[s] = f"{op}" + (f"({rd})" if rd else "")
        rows.append(row)
    df = pd.DataFrame(rows).set_index('cycle')
    csv_path = f"/content/pipeline/plots/{name}_forward{forwarding}_stall{stalling}_trace.csv"
    df.to_csv(csv_path)
    # Save as PNG table
    fig, ax = plt.subplots(figsize=(12, max(2, len(df)*0.25)))
    ax.axis('off')
    table = ax.table(cellText=df.reset_index().values, colLabels=df.reset_index().columns, loc='center', cellLoc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(8)
    table.scale(1, 1.2)
    png_path = f"/content/pipeline/plots/{name}_forward{forwarding}_stall{stalling}_timeline.png"
    plt.savefig(png_path, bbox_inches='tight', dpi=150)
    plt.close()
    # compute CPI
    cpi_stats = sim.compute_cpi()
    return {'df': df, 'csv': csv_path, 'png': png_path, 'cpi': cpi_stats, 'sim': sim}

results = {}
for k,v in examples.items():
    # simulate both with and without forwarding to show effect
    r1 = run_and_save(v, k, forwarding=True, stalling=True, predict_taken=False)
    r2 = run_and_save(v, k+"_nofwd", forwarding=False, stalling=True, predict_taken=False)
    results[k] = (r1, r2)
    print(f"Example {k}: saved {r1['png']} and {r2['png']} (CPI forward {r1['cpi']}, nofwd {r2['cpi']})")


Example load_use: saved /content/pipeline/plots/load_use_forwardTrue_stallTrue_timeline.png and /content/pipeline/plots/load_use_nofwd_forwardFalse_stallTrue_timeline.png (CPI forward {'total_instructions': 3, 'total_cycles': 9, 'CPI': 3.0}, nofwd {'total_instructions': 3, 'total_cycles': 12, 'CPI': 4.0})
Example alu_forward: saved /content/pipeline/plots/alu_forward_forwardTrue_stallTrue_timeline.png and /content/pipeline/plots/alu_forward_nofwd_forwardFalse_stallTrue_timeline.png (CPI forward {'total_instructions': 3, 'total_cycles': 8, 'CPI': 2.6666666666666665}, nofwd {'total_instructions': 3, 'total_cycles': 12, 'CPI': 4.0})
Example branch_case: saved /content/pipeline/plots/branch_case_forwardTrue_stallTrue_timeline.png and /content/pipeline/plots/branch_case_nofwd_forwardFalse_stallTrue_timeline.png (CPI forward {'total_instructions': 5, 'total_cycles': 10, 'CPI': 2.0}, nofwd {'total_instructions': 5, 'total_cycles': 12, 'CPI': 2.4})
Example chain: saved /content/pipeline/plots/

In [5]:
# Cell 5 - Draw simple pipeline diagram and forwarding unit diagram
def draw_pipeline_diagram(path="/content/pipeline/plots/pipeline_diagram.png"):
    fig, ax = plt.subplots(figsize=(10,2))
    ax.axis('off')
    stages = ['IF', 'ID', 'EX', 'MEM', 'WB']
    xs = [i*2 for i in range(len(stages))]
    for x,s in zip(xs, stages):
        ax.add_patch(plt.Rectangle((x,0), 1.6, 1, edgecolor='black', facecolor='#cfe2f3'))
        ax.text(x+0.8, 0.5, s, ha='center', va='center', fontsize=12, fontweight='bold')
        if x != xs[-1]:
            ax.annotate('', xy=(x+1.6,0.5), xytext=(x+2,0.5), arrowprops=dict(arrowstyle='->'))
    ax.set_xlim(-0.5, xs[-1]+2.5)
    ax.set_ylim(-0.5,1.6)
    plt.title("5-Stage Pipeline (IF -> ID -> EX -> MEM -> WB)")
    plt.savefig(path, bbox_inches='tight', dpi=150)
    plt.close()
    return path

def draw_forwarding_unit(path="/content/pipeline/plots/forwarding_unit.png"):
    fig, ax = plt.subplots(figsize=(6,4))
    ax.axis('off')
    ax.text(0.5,0.9,"Forwarding Unit", ha='center', fontsize=14, fontweight='bold')
    # draw EX/MEM/WB boxes and arrows to EX input
    ax.add_patch(plt.Rectangle((0.05,0.55),0.25,0.2,edgecolor='black', facecolor='#fce5cd'))
    ax.text(0.175,0.65,"EX/MEM\n(ALU result)", ha='center')
    ax.add_patch(plt.Rectangle((0.38,0.55),0.25,0.2,edgecolor='black', facecolor='#f9cb9c'))
    ax.text(0.505,0.65,"MEM/WB\n(Latent result)", ha='center')
    ax.add_patch(plt.Rectangle((0.7,0.55),0.25,0.2,edgecolor='black', facecolor='#d9ead3'))
    ax.text(0.825,0.65,"EX\n(ALU input)", ha='center')
    # arrows
    ax.annotate('', xy=(0.3,0.65), xytext=(0.7,0.75), arrowprops=dict(arrowstyle='->'))
    ax.annotate('', xy=(0.63,0.65), xytext=(0.7,0.7), arrowprops=dict(arrowstyle='->'))
    plt.title("Forwarding Paths into EX Stage")
    plt.savefig(path, bbox_inches='tight', dpi=150)
    plt.close()
    return path

p1 = draw_pipeline_diagram()
p2 = draw_forwarding_unit()
print("Saved diagrams:", p1, p2)


Saved diagrams: /content/pipeline/plots/pipeline_diagram.png /content/pipeline/plots/forwarding_unit.png


In [6]:
# Cell 6 - Create PDFs: hazard_handling_summary.pdf and pipeline_report.pdf
summary_text = textwrap.dedent("""
Hazard Handling Summary

This pipeline simulator supports:
- Data hazards: RAW hazards are detected in the ID stage. With forwarding enabled,
  most ALU-ALU hazards are resolved by forwarding data from EX/MEM or MEM/WB to EX input.
  Load-use hazards (LW followed by a dependent instruction) need a one-cycle stall if the load is in EX stage.
- Control hazards: Branches (BEQ) are resolved in EX stage. We use static branch prediction (predict_not_taken or predict_taken toggled).
  On misprediction the IF and ID stages are flushed (NOPs inserted) and PC set to branch target.
- Stalling: Implemented by inserting NOP in EX stage and freezing IF/ID fetch.
- Forwarding: When enabled, reduces stalls by directly supplying ALU inputs.
""")

# Save textual summary to PDF via matplotlib
pdf_path = "/content/pipeline/hazard_handling_summary.pdf"
with PdfPages(pdf_path) as pdf:
    fig = plt.figure(figsize=(8.5,11))
    plt.axis('off')
    plt.text(0.01, 0.99, "Hazard Handling Summary\n", fontsize=14, weight='bold', va='top')
    plt.text(0.01, 0.95, summary_text, fontsize=10, va='top')
    # include diagrams
    try:
        img1 = plt.imread('/content/pipeline/plots/pipeline_diagram.png')
        plt.figimage(img1, xo=50, yo=350, zorder=1)
    except:
        pass
    pdf.savefig()
    plt.close()

# Combined pipeline_report.pdf : include timelines, diagrams, CPI table
report_path = "/content/pipeline/pipeline_report.pdf"
with PdfPages(report_path) as pdf:
    # Page 1: title + diagrams
    fig = plt.figure(figsize=(8.5,11)); plt.axis('off')
    plt.text(0.01,0.98,"5-Stage Pipeline Report", fontsize=16, weight='bold', va='top')
    plt.text(0.01,0.93,"Generated by pipeline simulator", fontsize=10, va='top')
    try:
        img = plt.imread('/content/pipeline/plots/pipeline_diagram.png')
        plt.figimage(img, xo=60, yo=500, zorder=1)
    except:
        pass
    pdf.savefig(); plt.close()

    # Page 2: include one example timeline PNG
    fig = plt.figure(figsize=(8.5,11)); plt.axis('off')
    plt.text(0.01,0.98,"Example timeline (load-use, forwarding ON)", fontsize=12, weight='bold', va='top')
    try:
        img = plt.imread(results['load_use'][0]['png'])
        plt.figimage(img, xo=40, yo=100, zorder=1)
    except:
        pass
    pdf.savefig(); plt.close()
print("Saved PDFs:", pdf_path, report_path)


Saved PDFs: /content/pipeline/hazard_handling_summary.pdf /content/pipeline/pipeline_report.pdf


In [7]:
# Cell 7 - Compile CPI comparisons and plot
rows = []
for k, (r_forward, r_nofwd) in results.items():
    rows.append({'scenario': k, 'forwarding': True, 'CPI': r_forward['cpi']['CPI']})
    rows.append({'scenario': k, 'forwarding': False, 'CPI': r_nofwd['cpi']['CPI']})
df_cpi = pd.DataFrame(rows)
csv_cpi = '/content/pipeline/plots/cpi_comparison.csv'
df_cpi.to_csv(csv_cpi, index=False)

plt.figure(figsize=(8,4))
sns.barplot(data=df_cpi, x='scenario', y='CPI', hue='forwarding')
plt.title('CPI Comparison: Forwarding vs No Forwarding')
plt.savefig('/content/pipeline/plots/CPI_comparison.png', bbox_inches='tight')
plt.close()
print("Saved CPI comparison plot and CSV.")


Saved CPI comparison plot and CSV.


In [8]:
# Cell 8 - List generated deliverables
files = {
    'pipeline_dir': '/content/pipeline',
    'plots_dir': '/content/pipeline/plots',
    'hazard_pdf': '/content/pipeline/hazard_handling_summary.pdf',
    'report_pdf': '/content/pipeline/pipeline_report.pdf',
    'cpi_csv': '/content/pipeline/plots/cpi_comparison.csv'
}
for k,v in files.items():
    print(k, "->", v, "exists:", os.path.exists(v))
print("\nYou can download these files from Colab left panel (Files).")


pipeline_dir -> /content/pipeline exists: True
plots_dir -> /content/pipeline/plots exists: True
hazard_pdf -> /content/pipeline/hazard_handling_summary.pdf exists: True
report_pdf -> /content/pipeline/pipeline_report.pdf exists: True
cpi_csv -> /content/pipeline/plots/cpi_comparison.csv exists: True

You can download these files from Colab left panel (Files).
